In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/monodtr-original-library/MonoDTR/requirement.txt
/kaggle/input/monodtr-original-library/MonoDTR/INSTALL.md
/kaggle/input/monodtr-original-library/MonoDTR/LICENSE
/kaggle/input/monodtr-original-library/MonoDTR/README.md
/kaggle/input/monodtr-original-library/MonoDTR/DATA.md
/kaggle/input/monodtr-original-library/MonoDTR/make.sh
/kaggle/input/monodtr-original-library/MonoDTR/config/config.py
/kaggle/input/monodtr-original-library/MonoDTR/scripts/train.py
/kaggle/input/monodtr-original-library/MonoDTR/scripts/imdb_precompute_3d.py
/kaggle/input/monodtr-original-library/MonoDTR/scripts/eval.py
/kaggle/input/monodtr-original-library/MonoDTR/scripts/_path_init.py
/kaggle/input/monodtr-original-library/MonoDTR/scripts/imdb_precompute_test.py
/kaggle/input/monodtr-original-library/MonoDTR/scripts/depth_gt_compute.py
/kaggle/input/monodtr-original-library/MonoDTR/visualDet3D/__init__.py
/kaggle/input/monodtr-original-library/MonoDTR/visualDet3D/utils/readme.md
/kaggle/input/monodt

In [2]:
!mkdir -p /kaggle/working/MonoDTR
!cp -r /kaggle/input/monodtr-original-library/MonoDTR /kaggle/working/MonoDTR

In [3]:
import sys
sys.path.append("/kaggle/working/MonoDTR/MonoDTR")  # Add parent directory  # Add parent directory

In [4]:
!ls -l /kaggle/working/MonoDTR/MonoDTR

total 44
drwxr-xr-x 2 root root 4096 Apr  8 14:35 config
-rw-r--r-- 1 root root  709 Apr  8 14:35 DATA.md
-rw-r--r-- 1 root root  840 Apr  8 14:35 INSTALL.md
drwxr-xr-x 2 root root 4096 Apr  8 14:35 launchers
-rw-r--r-- 1 root root 1072 Apr  8 14:35 LICENSE
-rw-r--r-- 1 root root  764 Apr  8 14:35 make.sh
-rw-r--r-- 1 root root 2055 Apr  8 14:35 README.md
-rw-r--r-- 1 root root  131 Apr  8 14:35 requirement.txt
drwxr-xr-x 2 root root 4096 Apr  8 14:35 resources
drwxr-xr-x 2 root root 4096 Apr  8 14:35 scripts
drwxr-xr-x 6 root root 4096 Apr  8 14:35 visualDet3D


In [5]:
!cat /kaggle/working/MonoDTR/MonoDTR/make.sh

#!/bin/bash
set -e
#CUDA_PATH=$(whereis cuda | cut -d ' ' -f 2) #cuda: /usr/local/cuda -> /usr/local/cuda
#CUDA_VER=$(cat $CUDA_PATH/version.txt | cut -d ' ' -f 3)
TORCH_VER=$(python3 -c "import torch;print(torch.__version__)")
CUDA_VER=$(python3 -c "import torch;print(torch.version.cuda)")

if [[ $CUDA_VER < "10.0" || $TORCH_VER < '1.3' ]] ; then 
    echo "The current version of pytorch/cuda is $TORCH_VER/$CUDA_VER which could be not compatible with deformable convolution, we will not compile DCN for now. As long as you do not init DCN instance in code, the code will run fine."
else
    pushd visualDet3D/networks/lib/ops/dcn
    sh make.sh
    rm -r build
    popd

    pushd visualDet3D/networks/lib/ops/iou3d
    sh make.sh
    rm -r build
    popd
fi


In [6]:
# Create the modified make.sh file in a writable directory
with open('/kaggle/working/MonoDTR/MonoDTR/make.sh', 'w') as file:
    file.write("""#!/bin/bash
set -e
TORCH_VER=$(python3 -c "import torch;print(torch.__version__)")
CUDA_VER=$(python3 -c "import torch;print(torch.version.cuda)")

if [[ $CUDA_VER < "10.0" || $TORCH_VER < '1.3' ]] ; then 
    echo "The current version of pytorch/cuda is $TORCH_VER/$CUDA_VER which could be not compatible with deformable convolution, we will not compile DCN for now. As long as you do not init DCN instance in code, the code will run fine."
else
    pushd /kaggle/working/MonoDTR/MonoDTR/visualDet3D/networks/lib/ops/dcn
    sh make.sh /kaggle/working/build_dcn
    rm -r build
    popd

    pushd /kaggle/working/MonoDTR/MonoDTR/visualDet3D/networks/lib/ops/iou3d
    sh make.sh /kaggle/working/build_iou3d
    rm -r build
    popd
fi
""")

In [7]:
!cat /kaggle/working/MonoDTR/MonoDTR/make.sh

#!/bin/bash
set -e
TORCH_VER=$(python3 -c "import torch;print(torch.__version__)")
CUDA_VER=$(python3 -c "import torch;print(torch.version.cuda)")

if [[ $CUDA_VER < "10.0" || $TORCH_VER < '1.3' ]] ; then 
    echo "The current version of pytorch/cuda is $TORCH_VER/$CUDA_VER which could be not compatible with deformable convolution, we will not compile DCN for now. As long as you do not init DCN instance in code, the code will run fine."
else
    pushd /kaggle/working/MonoDTR/MonoDTR/visualDet3D/networks/lib/ops/dcn
    sh make.sh /kaggle/working/build_dcn
    rm -r build
    popd

    pushd /kaggle/working/MonoDTR/MonoDTR/visualDet3D/networks/lib/ops/iou3d
    sh make.sh /kaggle/working/build_iou3d
    rm -r build
    popd
fi


In [8]:
# Make the script executable
!chmod +x kaggle/working/MonoDTR/MonoDTR/make.sh

# Run the modified script
!bash /kaggle/working/MonoDTR/MonoDTR/make.sh

chmod: cannot access 'kaggle/working/MonoDTR/MonoDTR/make.sh': No such file or directory
/kaggle/working/MonoDTR/MonoDTR/visualDet3D/networks/lib/ops/dcn /kaggle/working
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:416: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:426: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.2
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['

In [9]:
!ls -l /kaggle/working/MonoDTR/MonoDTR/visualDet3D/networks/lib/ops/dcn

total 14360
-rwxr-xr-x 1 root root 14667088 Apr  8 14:36 deform_conv_ext.cpython-310-x86_64-linux-gnu.so
-rw-r--r-- 1 root root    20379 Apr  8 14:35 deform_conv.py
-rw-r--r-- 1 root root        0 Apr  8 14:35 __init__.py
-rw-r--r-- 1 root root      391 Apr  8 14:35 make.sh
-rw-r--r-- 1 root root     6447 Apr  8 14:35 setup.py
drwxr-xr-x 3 root root     4096 Apr  8 14:35 src


In [10]:
import importlib.util

spec = importlib.util.find_spec("visualDet3D")
print("Module found at:", spec.origin if spec else "Not found")

Module found at: /kaggle/working/MonoDTR/MonoDTR/visualDet3D/__init__.py


In [11]:
import os

# Let's find visualDet3D manually
for root, dirs, files in os.walk("/kaggle/working"):
    if "visualDet3D" in dirs:
        print("✅ Found visualDet3D at:", os.path.join(root, "visualDet3D"))
        break

✅ Found visualDet3D at: /kaggle/working/MonoDTR/MonoDTR/visualDet3D


In [12]:
import sys

# Replace this with your actual result from above
sys.path.insert(0, "/kaggle/working/MonoDTR/MonoDTR")

# Confirm it’s added
print("🛣 sys.path[0]:", sys.path[0])

🛣 sys.path[0]: /kaggle/working/MonoDTR/MonoDTR


In [13]:
import os
# List the contents of the directory where visualDet3D is supposed to be
base_path = '/kaggle/working/MonoDTR/MonoDTR'
print("Contents of MonoDTR directory:")
print(os.listdir(base_path))

# List the contents of the visualDet3D directory
visual_det_path = os.path.join(base_path, 'visualDet3D')
print("\nContents of visualDet3D directory:")
print(os.listdir(visual_det_path))

Contents of MonoDTR directory:
['resources', 'config', 'INSTALL.md', 'visualDet3D', 'launchers', 'README.md', 'make.sh', 'scripts', 'requirement.txt', 'DATA.md', 'LICENSE']

Contents of visualDet3D directory:
['evaluator', '__init__.py', 'utils', 'networks', 'data']


In [14]:
print("Checking for __init__.py files:")
for dir_path in ['visualDet3D', 'visualDet3D/networks', 'visualDet3D/networks/lib', 'visualDet3D/networks/lib/ops']:
    print(f"Exists in {dir_path}: {os.path.isfile(os.path.join(base_path, dir_path, '__init__.py'))}")

Checking for __init__.py files:
Exists in visualDet3D: True
Exists in visualDet3D/networks: True
Exists in visualDet3D/networks/lib: True
Exists in visualDet3D/networks/lib/ops: True


In [15]:
# Step 2: Import PyTorch and check the version
import torch
print("PyTorch version:", torch.__version__)

# Step 3: Check the PyTorch library path
print("PyTorch library path:", torch.__file__)

# Step 4: If necessary, set the LD_LIBRARY_PATH (usually not needed)
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/lib' + ':' + os.environ.get('LD_LIBRARY_PATH', '')

# Step 5: Attempt to import the deform_conv_ext module
try:
    import deform_conv_ext
    print("Import successful!")
except ImportError as e:
    print("Import failed:", e)

PyTorch version: 2.5.1+cu121
PyTorch library path: /usr/local/lib/python3.10/dist-packages/torch/__init__.py
Import failed: No module named 'deform_conv_ext'
